# Generate JSONL of facts and fake context facts using PopQA

In [2]:
import pandas as pd
import json
import numpy as np
import random
from ast import literal_eval
import json
from datasets import load_dataset
import os
os.chdir('..')
print(os.getcwd())
from utils.io import write_dicts_one_by_one

random.seed(42)
np.random.seed(42)

In [4]:
ds = load_dataset("akariasai/PopQA")
df = ds["test"].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


### EDA

In [2]:
# Sample 10 rows from each prop and display subj, obj, and prop
for prop in df["prop"].unique():
    print(f"\n=== {prop} ===")
    prop_subset = df[df["prop"] == prop]
    sample_size = min(10, len(prop_subset))
    sampled_rows = prop_subset.sample(n=sample_size, random_state=42)
    print(sampled_rows[["subj", "obj", "prop"]].to_string(index=False))



=== occupation ===
                subj            obj       prop
       Scooter Braun talent manager occupation
   Virginia Haviland      librarian occupation
       Annie Beustes     politician occupation
          Ali Hillis          actor occupation
   Guy Joseph Bonnet  major general occupation
      Eliza Lawrence     politician occupation
         Alan Huckle       diplomat occupation
Charles Martin Smith          actor occupation
      Jörg A. Eggers   screenwriter occupation
          Jon Eikemo          actor occupation

=== place of birth ===
              subj          obj           prop
      Paul Braniff   Portaferry place of birth
Alexander Butlerov       Çistay place of birth
      Mike Gregory         Bath place of birth
 Christian Vaquero   Montevideo place of birth
 Alexandros Matsas       Athens place of birth
    Charles Plumer  Canons Park place of birth
       Bob Warlick      Hickory place of birth
       Terry Hunte Saint Philip place of birth
       James Bro

### Filtering/Preprocessing

In [3]:
# filter out capital of because that one is weird
df = df[df["prop"] != "capital of"]
df

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[""journo"",""journalists""]",http://www.wikidata.org/entity/Q6247345,http://www.wikidata.org/entity/Q1930187,John Mayne,Journalist,236,24952,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,"[""Eleanor McCutcheon Davis""]","[""graphic artist"",""animator"",""illustrator""]",http://www.wikidata.org/entity/Q5354261,http://www.wikidata.org/entity/Q1114448,Eleanor Davis,Cartoonist,317,9649,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[""Holland"",""the Netherlands"",""NL"",""NED"",""Neder...","[""Mokum"",""Amsterdam, NL"",""Amsterdam, Netherlan...",http://www.wikidata.org/entity/Q55,http://www.wikidata.org/entity/Q727,Netherlands,Amsterdam,416350,147710,What is the capital of Netherlands?,"[""The Hague"", ""Den Haag"", ""'s-Gravenhage"", ""Ha..."
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,"[""Celje city municipality""]",[],http://www.wikidata.org/entity/Q3441823,http://www.wikidata.org/entity/Q1012,City Municipality of Celje,Celje,335,4086,What is the capital of City Municipality of Ce...,"[""Celje""]"
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[""National Capital Region of Delhi"",""NCR""]","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q1353,http://www.wikidata.org/entity/Q987,Delhi,New Delhi,219227,128193,What is the capital of Delhi?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[""occupied Palestinian territories"",""OPT"",""Wes...","[""Yerushalayim"",""J'lem"",""Aelia Capitolina"",""Al...",http://www.wikidata.org/entity/Q407199,http://www.wikidata.org/entity/Q1218,Palestinian territories,Jerusalem,17545,136734,What is the capital of Palestinian territories?,"[""Jerusalem"", ""Yerushalayim"", ""J'lem"", ""Aelia ..."


In [4]:
# Remove rows where the answer is too ambiguous.
print(df["prop"].value_counts())
print(df[df["subj"] == "Pilot"].head())
print(df[["subj", "prop"]].value_counts())

# Filter out rows where the combination of subj and prop appears more than once
subj_prop_counts = df.groupby(["subj", "prop"]).size()
duplicated_pairs = subj_prop_counts[subj_prop_counts > 1].index
df_filtered = df.set_index(["subj", "prop"]).drop(duplicated_pairs).reset_index()
df = df_filtered
print("After filtering, subj-prop pairs should be unique:")
print(df[["subj", "prop"]].value_counts()) ### All should be 1
print(df[df["subj"] == "London"]) # make sure no duplicate subj-prop pairs

print("After filtering, obj-prop pairs need not be unique:")
print(df[["obj", "prop"]].value_counts()) # obj-prop pairs need not be unique
print(df[df["obj"] == "association football"]) 


prop
director          1999
screenwriter      1999
genre             1619
producer          1520
author            1514
composer           978
country            838
capital            645
place of birth     584
father             570
sport              547
occupation         532
religion           338
mother             187
color               34
Name: count, dtype: int64
           id   subj      prop               obj  subj_id  prop_id   obj_id  \
5763  5453595  Pilot  director  Francis Lawrence  2425271      526  1883124   
5805  5453492  Pilot  director        Peter Berg  2425233      526   607993   
5818  4577696  Pilot  director     Marc Buckland  2018998      526  1230313   
5932  2957316  Pilot  director     Phillip Noyce  1266576      526  1791491   
5957   149939  Pilot  director      Brett Ratner    60507      526  1052983   

                        s_aliases o_aliases  \
5763  ["Tales of the Red Thread"]        []   
5805                           []        []   
5818    

In [5]:
def is_bijective(df, col_a, col_b):
    # Check if A->B mapping is one-to-one
    a_to_b_unique = df.groupby(col_a)[col_b].nunique().max() == 1
    
    # Check if B->A mapping is one-to-one
    b_to_a_unique = df.groupby(col_b)[col_a].nunique().max() == 1
    
    # Both conditions must be true
    return a_to_b_unique and b_to_a_unique

# Example usage
result = is_bijective(df, 'o_uri', 'obj_id')
print(f"Columns are bijective: {result}")

result = is_bijective(df, 'o_aliases', 'possible_answers')
print(f"Columns are bijective: {result}")

Columns are bijective: True
Columns are bijective: False


In [6]:
# Filter out all obj_ids that have only 1 row
# Keep only obj_ids that appear more than once
# Count how many times each obj_id appears
obj_id_counts = df['obj_id'].value_counts()

# Get obj_ids that appear more than once
obj_ids_multiple = obj_id_counts[obj_id_counts > 1].index

# Filter the dataframe to keep only rows with obj_ids that appear multiple times
df_filtered = df[df['obj_id'].isin(obj_ids_multiple)]

print(f"Original dataframe shape: {df.shape}")
print(f"Filtered dataframe shape: {df_filtered.shape}")
print(f"Removed {df.shape[0] - df_filtered.shape[0]} rows with unique obj_ids")

# Show some statistics
print(f"\nOriginal obj_id counts:")
print(f"  Total unique obj_ids: {df['obj_id'].nunique()}")
print(f"  obj_ids with count = 1: {(obj_id_counts == 1).sum()}")
print(f"  obj_ids with count > 1: {(obj_id_counts > 1).sum()}")

print(f"\nFiltered obj_id counts:")
filtered_counts = df_filtered['obj_id'].value_counts()
print(f"  Total unique obj_ids: {df_filtered['obj_id'].nunique()}")
print(f"  Min count per obj_id: {filtered_counts.min()}")
print(f"  Max count per obj_id: {filtered_counts.max()}")

# Update the main dataframe
df = df_filtered


Original dataframe shape: (11984, 17)
Filtered dataframe shape: (7195, 17)
Removed 4789 rows with unique obj_ids

Original obj_id counts:
  Total unique obj_ids: 6402
  obj_ids with count = 1: 4789
  obj_ids with count > 1: 1613

Filtered obj_id counts:
  Total unique obj_ids: 1613
  Min count per obj_id: 2
  Max count per obj_id: 327


In [7]:
df.to_csv("data/popqa_filtered.csv", index=False)

### Construct facts JSONL

In [8]:
condition_data = df.sort_values(by="s_pop", ascending=False).head(100).reset_index(drop=True)
condition_data

,subj,prop,id,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,United States of America,capital,2266274,"Washington, D.C.",988513,422,2024419,"[""the United States of America"",""America"",""U.S...","[""Washington"",""Washington DC"",""Washington, DC""...",http://www.wikidata.org/entity/Q30,http://www.wikidata.org/entity/Q61,United States,"Washington, D.C.",1629691,283465,What is the capital of United States of America?,"[""Washington, D.C."", ""Washington"", ""Washington..."
1,India,capital,5015775,New Delhi,2215085,422,2938512,"[""Bharat"",""Hindustan"",""Bharatvarsh"",""in"",""IN"",...","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q668,http://www.wikidata.org/entity/Q987,India,New Delhi,1301086,128193,What is the capital of India?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
2,Elizabeth II,religion,6515620,Anglicanism,2925405,106,2145926,"[""Queen Elizabeth II"",""Elizabeth Alexandra Mar...","[""Anglicanism, Anglican Church""]",http://www.wikidata.org/entity/Q9682,http://www.wikidata.org/entity/Q6423963,Elizabeth II,Anglicanism,1291432,56744,What is the religion of Elizabeth II?,"[""Anglicanism"", ""Anglicanism, Anglican Church""]"
3,United Kingdom,capital,593981,London,242916,422,2840765,"[""\ud83c\uddec\ud83c\udde7"",""UK"",""United Kingd...","[""London, UK"",""London, United Kingdom"",""London...",http://www.wikidata.org/entity/Q145,http://www.wikidata.org/entity/Q84,United Kingdom,London,1066107,523018,What is the capital of United Kingdom?,"[""London"", ""London, UK"", ""London, United Kingd..."
4,Kanye West,occupation,809091,record producer,345517,22,620958,"[""Kanye Omari West"",""Yeezy"",""Ye"",""Yeezus"",""The...","[""music producer""]",http://www.wikidata.org/entity/Q15935,http://www.wikidata.org/entity/Q183945,Kanye West,Record producer,940772,42844,What is Kanye West's occupation?,"[""record producer"", ""music producer"", ""songwri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A.C. Milan,color,736578,red,313273,472,1036579,"[""AC Milan"",""Associazione Calcio Milan"",""Assoc...","[""red color""]",http://www.wikidata.org/entity/Q1543,http://www.wikidata.org/entity/Q3142,A.C. Milan,Red,188472,51772,What color is A.C. Milan?,"[""red"", ""red color""]"
96,Jurassic Park,screenwriter,1057843,David Koepp,453107,533,907664,[],[],http://www.wikidata.org/entity/Q167726,http://www.wikidata.org/entity/Q270178,Jurassic Park (film),David Koepp,184720,13666,Who was the screenwriter for Jurassic Park?,"[""Michael Crichton"", ""John Michael Crichton"", ..."
97,Lithuania,capital,2776008,Vilnius,1191903,422,752152,"[""Republic of Lithuania"",""Lietuva"",""\ud83c\udd...","[""Vilna"",""Wilno"",""Vilne"",""Wilna"",""Vi\u013c\u01...",http://www.wikidata.org/entity/Q37,http://www.wikidata.org/entity/Q216,Lithuania,Vilnius,183018,42978,What is the capital of Lithuania?,"[""Vilnius"", ""Vilna"", ""Wilno"", ""Vilne"", ""Wilna""..."
98,Wales,capital,1927140,Cardiff,847062,422,40007,"[""WAL"",""WLS"",""Cymru"",""Cambria""]","[""Caerdydd"",""Cardiff, Wales""]",http://www.wikidata.org/entity/Q25,http://www.wikidata.org/entity/Q10690,Wales,Cardiff,177730,50234,What is the capital of Wales?,"[""Cardiff"", ""Caerdydd"", ""Cardiff, Wales""]"


In [9]:
df["o_aliases"] = df["o_aliases"].apply(literal_eval)
df["possible_answers"] = df["possible_answers"].apply(literal_eval)
df["s_aliases"] = df["s_aliases"].apply(literal_eval)

In [10]:
facts = []
for i, row in df.iterrows():
    relation = row["prop"]
    possible_obj_ctxs = df[df["prop"] == relation]["obj"].unique()
    obj_ctx = row["obj"]
    obj_prior_possibilities = [row["obj"]] + row["o_aliases"]
    while obj_ctx in obj_prior_possibilities:
        obj_ctx = random.choice(possible_obj_ctxs)
        obj_ctx_aliases = df[df["obj"] == obj_ctx]["o_aliases"].iloc[0]
    extra_info_obj_ctxs_data = df[df["obj"] == obj_ctx]
    extra_info_obj_ctx_data = extra_info_obj_ctxs_data.sample(n=1)
    extra_info_obj_ctx = f"is the {extra_info_obj_ctx_data['prop'].item()} of {extra_info_obj_ctx_data['subj'].item()}"
    
    condition_row = condition_data.sample(n=1)
    condition = f"{condition_row['obj'].item()} is the {condition_row['prop'].item()} of {condition_row['subj'].item()}"
    counterfactual_condition = f"{condition_row['obj'].item()} were the {condition_row['prop'].item()} of {condition_row['subj'].item()}"
    facts.append(
        {
            "subject": row["subj"],
            "relation": row["prop"],
            "object_ctx": obj_ctx,
            "object_pri": row["obj"],
            "object_ctx_aliases": obj_ctx_aliases,
            "object_prior_aliases": row["o_aliases"],
            "possible_prior_answers": row["possible_answers"],
            "obj_prior_id": row["obj_id"],
            "subj_pop": row["s_pop"],
            "obj_pop": row["o_pop"],
            "subject_relation": f"the {row['prop']} of {row['subj']}",
            "extra_info_obj_ctx": extra_info_obj_ctx,
            "condition": condition,
            "counterfactual_condition": counterfactual_condition,
            "condition_index": int(condition_row.index[0])
        }
    )


In [11]:
facts[:10]

[{'subject': 'George Rankin',
  'relation': 'occupation',
  'object_ctx': 'biologist',
  'object_pri': 'politician',
  'object_ctx_aliases': ['biologists'],
  'object_prior_aliases': ['political leader',
   'political figure',
   'polit.',
   'pol'],
  'possible_prior_answers': ['politician',
   'political leader',
   'political figure',
   'polit.',
   'pol'],
  'obj_prior_id': 2834605,
  'subj_pop': 142,
  'obj_pop': 25692,
  'subject_relation': 'the occupation of George Rankin',
  'extra_info_obj_ctx': 'is the occupation of Jean-Baptiste de Lamarck',
  'condition': 'Rob Zombie is the director of Halloween II',
  'counterfactual_condition': 'Rob Zombie were the director of Halloween II',
  'condition_index': 83},
 {'subject': 'John Mayne',
  'relation': 'occupation',
  'object_ctx': 'lawyer',
  'object_pri': 'journalist',
  'object_ctx_aliases': ['attorney', 'solicitor', 'barrister', 'lawyers'],
  'object_prior_aliases': ['journo', 'journalists'],
  'possible_prior_answers': ['journa

In [12]:
write_dicts_one_by_one(facts, "data/popqa_filtered.jsonl")

Writing 7195 records to data/popqa_filtered.jsonl (one by one)
Error writing item: 'utf-8' codec can't encode characters in position 164-167: surrogates not allowed
{'subject': 'Alastair Heathcote', 'relation': 'place of birth', 'object_ctx': 'Uganda', 'object_pri': 'Athens', 'object_ctx_aliases': ['Republic of Uganda', 'ug', '\ud83c\uddfa\ud83c\uddec', 'UGA'], 'object_prior_aliases': [], 'possible_prior_answers': ['Athens'], 'obj_prior_id': 299930, 'subj_pop': 161, 'obj_pop': 101288, 'subject_relation': 'the place of birth of Alastair Heathcote', 'extra_info_obj_ctx': 'is the place of birth of Louis Kasekende', 'condition': 'The Wachowskis is the director of The Matrix', 'counterfactual_condition': 'The Wachowskis were the director of The Matrix', 'condition_index': 52}
Error writing item: 'utf-8' codec can't encode characters in position 172-175: surrogates not allowed
{'subject': 'Austin Cuvillier', 'relation': 'place of birth', 'object_ctx': 'Egypt', 'object_pri': 'Quebec City', 'o

### Scratch

In [13]:
# # Find rows where o_aliases and possible_answers are not bijective
# print("=== ANALYZING NON-BIJECTIVE RELATIONSHIPS ===\n")

# # Check if columns exist
# if 'o_aliases' not in df.columns:
#     print("Error: 'o_aliases' column not found in dataframe")
#     print("Available columns:", df.columns.tolist())
# elif 'possible_answers' not in df.columns:
#     print("Error: 'possible_answers' column not found in dataframe") 
#     print("Available columns:", df.columns.tolist())
# else:
#     # First, let's check overall bijection status
#     result = is_bijective(df, 'o_aliases', 'possible_answers')
#     print(f"Overall bijection status: {result}")
    
#     if not result:
#         print("\n=== FINDING BIJECTION VIOLATIONS ===\n")
        
#         # Convert to string representation for grouping (since these might be lists)
#         df_str = df.copy()
#         df_str['o_aliases_str'] = df_str['o_aliases'].astype(str)
#         df_str['possible_answers_str'] = df_str['possible_answers'].astype(str)
        
#         # Check o_aliases -> possible_answers violations (one o_aliases maps to multiple possible_answers)
#         alias_to_answers = df_str.groupby('o_aliases_str')['possible_answers_str'].nunique()
#         alias_violations = alias_to_answers[alias_to_answers > 1]
        
#         if len(alias_violations) > 0:
#             print(f"1. ALIASES -> ANSWERS VIOLATIONS: {len(alias_violations)} aliases map to multiple answer sets")
#             print("   (Same alias appears with different possible_answers)\n")
            
#             for alias_str in alias_violations.index[:5]:  # Show first 5
#                 matching_rows = df_str[df_str['o_aliases_str'] == alias_str]
#                 print(f"   Alias: {matching_rows.iloc[0]['o_aliases']}")
#                 print(f"   Maps to {len(matching_rows)} different answer sets:")
#                 for i, (_, row) in enumerate(matching_rows.iterrows()):
#                     print(f"     {i+1}. {row['possible_answers']} (subj: {row['subj']}, prop: {row['prop']})")
#                 print()
        
#         # Check possible_answers -> o_aliases violations (one possible_answers maps to multiple o_aliases)
#         answers_to_alias = df_str.groupby('possible_answers_str')['o_aliases_str'].nunique()
#         answer_violations = answers_to_alias[answers_to_alias > 1]
        
#         if len(answer_violations) > 0:
#             print(f"2. ANSWERS -> ALIASES VIOLATIONS: {len(answer_violations)} answer sets map to multiple aliases")
#             print("   (Same possible_answers appears with different o_aliases)\n")
            
#             for answers_str in answer_violations.index[:5]:  # Show first 5
#                 matching_rows = df_str[df_str['possible_answers_str'] == answers_str]
#                 print(f"   Possible answers: {matching_rows.iloc[0]['possible_answers']}")
#                 print(f"   Maps to {len(matching_rows)} different aliases:")
#                 for i, (_, row) in enumerate(matching_rows.iterrows()):
#                     print(f"     {i+1}. {row['o_aliases']} (subj: {row['subj']}, prop: {row['prop']})")
#                 print()
        
#         # Show some sample problematic rows
#         print("=== SAMPLE PROBLEMATIC ROWS ===\n")
        
#         if len(alias_violations) > 0:
#             sample_alias = alias_violations.index[0]
#             problem_rows = df_str[df_str['o_aliases_str'] == sample_alias]
#             print("Sample rows with same o_aliases but different possible_answers:")
#             print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
#             print()
        
#         if len(answer_violations) > 0:
#             sample_answer = answer_violations.index[0]
#             problem_rows = df_str[df_str['possible_answers_str'] == sample_answer]
#             print("Sample rows with same possible_answers but different o_aliases:")
#             print(problem_rows[['subj', 'obj', 'prop', 'o_aliases', 'possible_answers']].to_string())
    
#     else:
#         print("The columns are bijective - no violations found!")
